In [1]:
# Instala as bibliotecas necessárias no container
import sys
# A flag --force-reinstall garante que ele troque a versão nova pela antiga compatível
# Fixamos o numpy em 1.24.3 para satisfazer o Numba e o Scikit-Learn
!{sys.executable} -m pip install opencv-python-headless requests matplotlib "numpy==1.24.3" --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def mostrar(img, titulo="Imagem"):
    plt.figure(figsize=(10, 6))
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap='gray')
    plt.title(titulo)
    plt.axis('off')
    plt.show()

print("🛠️ Gerando Imagem Simulada v2 (Calibrada)...")

# 1. Fundo Cinza (Telhado)
img_original = np.zeros((600, 800, 3), dtype=np.uint8)
img_original[:] = (180, 180, 180) 

# 2. Painéis Solares (Menores para passar no filtro)
# Cor Azul Escuro BGR: (Blue=160, Green=50, Red=40)
cor_painel = (160, 50, 40) 

# Painel 1 (Tamanho: 100x150 = 15.000 pixels -> PERFEITO)
cv2.rectangle(img_original, (150, 200), (250, 350), cor_painel, -1)
cv2.rectangle(img_original, (150, 200), (250, 350), (200, 200, 200), 3) # Borda

# Painel 2 (Tamanho: 100x150 = 15.000 pixels -> PERFEITO)
cv2.rectangle(img_original, (500, 200), (600, 350), cor_painel, -1)
cv2.rectangle(img_original, (500, 200), (600, 350), (200, 200, 200), 3) # Borda

# 3. Ruído (Piscina Azul Claro - Deve ser ignorada pela cor)
cv2.rectangle(img_original, (350, 450), (450, 550), (255, 200, 100), -1)

print("✅ Imagem pronta.")
mostrar(img_original, "Imagem Simulada")

In [ ]:
# Pré-processamento
hsv = cv2.cvtColor(img_original, cv2.COLOR_BGR2HSV)

# Intervalo de Cor "Azul Solar" (Expandido para garantir detecção)
# Matiz (H): 90 a 140 (Azul puro até violeta)
lower_blue = np.array([90, 40, 30])
upper_blue = np.array([140, 255, 255])

mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Limpeza
kernel = np.ones((5,5), np.uint8)
mask_clean = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# Detecção
contours, _ = cv2.findContours(mask_clean, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

img_resultado = img_original.copy()
area_total_pixels = 0
paineis_encontrados = 0

for cnt in contours:
    area = cv2.contourArea(cnt)
    
    # Filtro de Tamanho: 
    # Aceita qualquer coisa entre 1.000px e 100.000px
    if 1000 < area < 100000:
        
        # Simplificação geométrica (foca em retângulos)
        perimetro = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * perimetro, True)
        
        # Se tiver 4 vértices, desenha
        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            # Desenha em VERDE BRILHANTE
            cv2.rectangle(img_resultado, (x, y), (x+w, y+h), (0, 255, 0), 4)
            cv2.putText(img_resultado, f"Solar {area:.0f}px", (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            
            area_total_pixels += area
            paineis_encontrados += 1

mostrar(mask_clean, "Máscara (Visão do Robô)")
mostrar(img_resultado, f"Resultado ({paineis_encontrados} detecções)")

In [ ]:
# --- SIMULAÇÃO DE AUDITORIA ---

# Conversão Pixels -> Metros Quadrados (Depende do zoom do satélite)
# Vamos chutar que cada pixel é 0.05 m² nessa imagem
fator_conversao = 0.005 
area_m2_detectada = area_total_pixels * fator_conversao

# Potência Estimada (1 m² de painel moderno ~= 200W ou 0.2kW)
potencia_visual_kw = area_m2_detectada * 0.2

# Dado do Banco de Dados (Simulado: ANEEL diz que ali não tem nada)
potencia_aneel_kw = 0.0 

print(f"--- RELATÓRIO DE INSPEÇÃO ---")
print(f"👁️ Área Detectada (IA): {area_m2_detectada:.2f} m²")
print(f"⚡ Potência Estimada: {potencia_visual_kw:.2f} kW")
print(f"📝 Registro ANEEL: {potencia_aneel_kw:.2f} kW")
print("-" * 30)

diferenca = potencia_visual_kw - potencia_aneel_kw

if diferenca > 1.0:
    print(f"🚨 ALERTA: Potencial Carga Oculta de {diferenca:.2f} kW!")
else:
    print("✅ Regular.")

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
import random

# --- CONFIGURAÇÃO ---
DB_URL = "postgresql://admin:admin123@db:5432/energy_monitor"
engine = create_engine(DB_URL)

print("🔄 SINCRONIZANDO COM O BANCO DE DADOS...")

# ==============================================================================
# 1. BUSCAR NOMES REAIS DAS DISTRIBUIDORAS (Para bater com o Sidebar)
# ==============================================================================
# Consultamos as distribuidoras que realmente têm dados no sistema
with engine.connect() as conn:
    # Pega as 5 maiores distribuidoras (mesma lógica que o sidebar usa)
    query = text("""
        SELECT distribuidora 
        FROM gd_detalhada 
        GROUP BY distribuidora 
        ORDER BY SUM(potencia_mw) DESC 
        LIMIT 1000
    """)
    result = conn.execute(query).fetchall()
    
    # Cria uma lista limpa com os nomes exatos. Ex: ['CEMIG DISTRIBUIÇÃO S.A', 'ENEL SP'...]
    lista_distribuidoras_reais = [row[0] for row in result]

# FALLBACK: Se o banco estiver vazio (você ainda não rodou o ETL), usa nomes genéricos
if not lista_distribuidoras_reais:
    print("⚠️ Banco de dados de GD parece vazio. Usando nomes padrão para teste.")
    lista_distribuidoras_reais = ["ENEL", "CEMIG", "LIGHT", "COPEL", "CPFL"]

print(f"✅ Alvos identificados: {lista_distribuidoras_reais}")

# ==============================================================================
# 2. GERAÇÃO DA IMAGEM E VISÃO COMPUTACIONAL
# ==============================================================================
# (Mesmo código de processamento de imagem de antes)
img_original = np.zeros((600, 800, 3), dtype=np.uint8)
img_original[:] = (180, 180, 180) 
cor_painel = (160, 50, 40)
cv2.rectangle(img_original, (150, 200), (250, 350), cor_painel, -1)
cv2.rectangle(img_original, (500, 200), (600, 350), cor_painel, -1)

# Detecta
hsv = cv2.cvtColor(img_original, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, np.array([90, 40, 30]), np.array([140, 255, 255]))
contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

area_pixels = sum([cv2.contourArea(cnt) for cnt in contours if cv2.contourArea(cnt) > 1000])
area_base_m2 = area_pixels * 0.01
kw_base = area_base_m2 * 0.2

# ==============================================================================
# 3. GERAR FRAUDES PARA A LISTA EXATA DO DROPDOWN
# ==============================================================================
auditorias_para_salvar = []
print("\n🛰️ GERANDO RELATÓRIOS DE AUDITORIA...")

# Função auxiliar de classificação
def classificar(area):
    if area < 80: return "Residencial"
    elif area < 600: return "Comercial"
    else: return "Industrial"

# Loop pelos nomes REAIS do banco
for i, nome_distribuidora in enumerate(lista_distribuidoras_reais):
    
    # LÓGICA DE DIVERSIDADE DA DEMO:
    # A 1ª da lista (Maior) -> Simula fraude INDUSTRIAL (Gigante)
    # A 2ª da lista -> Simula fraude COMERCIAL (Média)
    # As outras -> Simulam fraude RESIDENCIAL (Pequena)
    
    if i == 0: 
        fator = 30.0 # Industrial
        tipo_simulado = "Industrial"
    elif i == 1:
        fator = 5.0  # Comercial
        tipo_simulado = "Comercial"
    else:
        fator = 1.0  # Residencial
        tipo_simulado = "Residencial"

    # Calcula valores finais
    area_final = area_base_m2 * fator
    kw_final = kw_base * fator
    
    # Define classe
    classe_ia = classificar(area_final)
    
    # Cria o registro
    auditoria = {
        "data_inspecao": datetime.now(),
        "latitude": -23.55 + (i * 0.5), # Varia um pouco a latitude para não ficar tudo no mesmo ponto
        "longitude": -46.63,
        "distribuidora": nome_distribuidora, # <--- AQUI ESTÁ O NOME EXATO DO DROPDOWN
        "classe_estimada_ia": classe_ia,
        "area_detectada_m2": area_final,
        "potencia_estimada_kw": kw_final,
        "potencia_oficial_kw": 0.0,
        "diferenca_fraude_kw": kw_final,
        "status": "ALERTA"
    }
    
    auditorias_para_salvar.append(auditoria)
    print(f"   -> Gerado para '{nome_distribuidora}': {tipo_simulado} ({kw_final:.1f} kW)")

# ==============================================================================
# 4. SALVAR NO BANCO
# ==============================================================================
df_audit = pd.DataFrame(auditorias_para_salvar)

with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS auditoria_visual"))
    conn.commit()

try:
    df_audit.to_sql('auditoria_visual', engine, if_exists='append', index=False)
    print("\n✅ SUCESSO! Dados sincronizados com o Sidebar.")
except Exception as e:
    print(f"❌ Erro: {e}")